In [26]:
# discharge.py usage
# by Rui, Aug 2021

In [33]:
import pickle
import src.discharge

In [32]:
"""
inputs:
    ----------
    reach:
        - reach['area_fits'] contains the parameters of cross-sectional area change  
        - reach['discharge_models'] contains the parameters of 5 discharge models  
        
    reach_stats: 
        - reach_stats['height'], SWOT reach height [m] 
        - reach_stats['width'], SWOT reach width [m]
        - reach_stats['slope'], SWOT reach slope [m/m]
        
outputs:
    ----------
    d_x_area: cross-sectional area change [m^2]
    d_x_area_u: cross-sectional area change uncertainty [m^2]
    metro_q_c : MetroMan discharge gage constrained [m^3/s]
    metro_q_uc : MetroMan discharge gage un-constrained [m^3/s]
    bam_q_c : BAM discharge gage constrained [m^3/s]
    bam_q_uc : BAM discharge gage un-constrained [m^3/s]
    bam_q_c : BAM discharge gage constrained [m^3/s]
    bam_q_uc : BAM discharge gage un-constrained [m^3/s]
    hivdi_q_c : HiVDI discharge gage constrained [m^3/s]
    hivdi_q_uc : HiVDI discharge gage un-constrained [m^3/s]
    momma_q_c : MOMMA discharge gage constrained [m^3/s]
    momma_q_uc : MOMMA discharge gage un-constrained [m^3/s]
    sads_q_c : SADS discharge gage constrained [m^3/s]
    sads_q_uc : SADS discharge gage un-constrained [m^3/s]
    
    **Discharge uncertainties are not implemented at this moment
"""

# load data saved from the RiverObs run on Sac reaches
# note: some of the items under the dict provided in the /data won't be used
reach_stats = pickle.load(open
                ('/Users/rui/Desktop/Github/offline-discharge-data-product-creation/data/reach25.pkl', "rb"))
reach = pickle.load(open
                ('/Users/rui/Desktop/Github/offline-discharge-data-product-creation/data/reach_prd25.pkl', "rb"))

In [30]:
# run discharge module
# copied from RiverObs/src/SWOTRiver
discharge_model_values = src.discharge.compute(
            reach, reach_stats['height'], reach_stats['width'],
            reach_stats['slope'])


print(discharge_model_values)

{'d_x_area': masked_array(data=52.33427927,
             mask=False,
       fill_value=-9999.0), 'd_x_area_u': 11.824505731801548, 'metro_q_uc': -999999999999, 'bam_q_uc': -999999999999, 'hivdi_q_uc': -999999999999, 'momma_q_uc': -999999999999, 'sads_q_uc': -999999999999, 'metro_q_c': masked_array(data=[238.38807826039445],
             mask=[False],
       fill_value=-9999.0), 'bam_q_c': masked_array(data=[213.30025300914113],
             mask=[False],
       fill_value=-9999.0), 'hivdi_q_c': masked_array(data=[238.3870520470281],
             mask=[False],
       fill_value=-9999.0), 'momma_q_c': masked_array(data=[215.14117112199818],
             mask=[False],
       fill_value=-9999.0), 'sads_q_c': masked_array(data=[213.30025300914113],
             mask=[False],
       fill_value=-9999.0)}
